In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import auc
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize
import collections
import seaborn as sns
import matplotlib.pyplot as plt
import math
import scipy.stats as sps
import difflib as dl
%matplotlib inline

In [22]:
data = pd.read_csv('task2_lemmas_train').fillna('')
test = pd.read_csv('task2_lemmas_test', index_col=0).fillna('')

Если для одного слова есть несколько ответов, то добавлю их в y

In [23]:
olddata = data.copy()
newdata = data
for col in ['y1','y2','y3','y4']:
    newdata = newdata.select(lambda x: len(newdata[col][x]) > 0)
    newdata['y'] = newdata[col]
    data = data.append(newdata)

In [24]:
data = data[['X','y']]
data['PoS'] = data['y'].map(lambda x: x[-1:])
data['y'] = data['y'].map(lambda x: x[:-2])
mem_data = data.copy()
data.head()

,X,y,PoS
0,vergognerete,vergognare,V
1,amnistiavate,amnistiare,V
2,menomazione,menomazione,N
3,sfaldavamo,sfaldare,V
4,sfodererei,sfoderare,V


Составим множество возможных окончаний

In [71]:
exp_data = data.copy()
exp_data['X_beg'] = exp_data['X_end'] = exp_data['y_end'] = exp_data['X']

for x in exp_data.iterrows():
    i = 0
    while(x[1]['X'][i] == x[1]['y'][i]):
        i+=1
        if(i == len(x[1]['y']) or i == len(x[1]['X'])):
            x[1]['X_end'] = x[1]['y'] = ''
            x[1]['X_beg'] = x[1]['X']
            break
    
    if(i < len(x[1]['y'])):
        x[1]['X_beg'] = x[1]['X'][:i]
        x[1]['X_end'] = x[1]['X'][i:]
        x[1]['y_end'] = x[1]['y'][i:]

In [6]:
def sortByLength(inputStr):
        return len(inputStr)
endings = np.unique(exp_data[['X_end', 'y']].as_matrix().reshape(-1)).tolist()
endings.sort(key=sortByLength)

In [ ]:
var = exp_data.copy()
#exp_data = var.copy()
exp_data['X_end_new'] = exp_data['X'].map(lambda x: [x])
for ending in endings:
    exp_data['X_end_new'] = exp_data['X_end_new'].map(lambda x: x + [ending] if len(x[0]) >= len(ending) and x[0][-len(ending):] == ending else x)
exp_data.head()

In [99]:
#arr_data = exp_data.copy()
exp_data['X_end_new'] = exp_data['X_end_new'].map(lambda x: x if len(x)>1 and x[0]!=x[-1] else x + [x[0][-1]])
exp_data['X_beg'] = exp_data['X_end_new'].map(lambda x: x[0][:-len(x[-1])])
exp_data['X_end_new'] = exp_data['X_end_new'].map(lambda x: x[-1] if len(x)>1 else x[0][-1])
exp_data.head()

,X,y,PoS,X_beg,X_end,y_end,X_end_new
37,balenieri,baleniero,A,balenier,i,o,i
48,osanti,osare,A,osa,nti,re,nti
49,rialzata,rialzato,A,rialzat,a,o,a
55,rivaleggiante,rivaleggiare,A,rivaleggia,nte,re,nte
61,cospiranti,cospirare,A,cospira,nti,re,nti


In [100]:
exp_data['X_end'] = exp_data['X_end_new']
exp_data['y_beg'] = exp_data['PoS']
for x in exp_data.iterrows():
    x[1]['y_end'] = x[1]['y'][len(x[1]['X_beg']):]
    x[1]['y_beg'] = x[1]['y'][:len(x[1]['X_beg'])]
exp_data.head()

,X,y,PoS,X_beg,X_end,y_end,X_end_new,y_beg
37,balenieri,baleniero,A,balenier,i,o,i,balenier
48,osanti,osare,A,osa,nti,re,nti,osa
49,rialzata,rialzato,A,rialzat,a,o,a,rialzat
55,rivaleggiante,rivaleggiare,A,rivaleggia,nte,re,nte,rivaleggia
61,cospiranti,cospirare,A,cospira,nti,re,nti,cospira


In [ ]:
exp_data['PoS'].unique()

In [ ]:
arr_data.loc[exp_data['X_beg'] != exp_data['y_beg']]

In [ ]:
exp_data.loc[exp_data['X_beg'] != exp_data['y_beg']]

1. Предсказать часть речи
2. Отделять не только суффиксы/окончания, но и приставки
3. А что если они пустые? - рассматривать их отдельно и думать

In [9]:
count_vectorizer = CountVectorizer(min_df=5, max_df=.5, max_features=None,ngram_range=(2, 5), analyzer='char_wb', lowercase=False, binary=True) 
X_train_pos = count_vectorizer.fit_transform(data['X'])
y_train_pos = data['PoS']
#y_train_pos = label_binarize(y_train_pos, classes=['N', 'A', 'V'])
X_test_pos = count_vectorizer.transform(test['X'])

In [65]:
algo = LogisticRegression(penalty='l1', C=0.145)
arr = cross_val_score(algo, X_train_pos, y_train_pos, cv=5)
arr

array([ 0.92377998,  0.92303557,  0.92381819,  0.92923611,  0.894123  ])

In [ ]:
res = []
grid = np.linspace(1., 10., 4)
for c in grid:
    algo = LogisticRegression(penalty='l1', C=c)
    res += [cross_val_score(algo, X_train_pos, y_train_pos, cv=5, scoring='accuracy')]

In [ ]:
res = np.array(res)
plt.figure()
plt.plot(grid, res.mean(axis=1), color='green')
plt.plot(grid, res.max(axis=1), color='red')
plt.plot(grid, res.min(axis=1), color='yellow')
plt.show

In [69]:
algo = LogisticRegression(penalty='l1', C=4)
algo = algo.fit(X_train_pos, y_train_pos)
test['PoS'] = algo.predict(X_test_pos)
test

,X,y,PoS
1,gettonan,gettonare+V,V
2,incidentali,incidentali,A
3,involtino,involtinare+V,V
4,lievi,lievi,N
5,comunistizzasse,comunistizzare+V,V
6,vidimerebbe,vidimeare+V,V
7,imbrodan,imbrodare+V,V
8,strillar,strillare+V,V
9,cifrasti,cifrasre+V,V
10,compassavano,compassare+V,V


In [102]:
#test['y'] = test['PoS']
#arr_data = exp_data.copy()
train_v = arr_data.copy().loc[arr_data['PoS']=='A']
test_v = test.copy().loc[test['PoS']=='A']
train_v

,X,y,PoS,X_beg,X_end,y_end
37,balenieri,baleniero,A,balenier,i,o
48,osanti,osare,A,osa,nti,re
49,rialzata,rialzato,A,rialzat,a,o
55,rivaleggiante,rivaleggiare,A,rivaleggia,nte,re
61,cospiranti,cospirare,A,cospira,nti,re
81,turbolenta,turbolento,A,turbolent,a,o
83,vaticinante,vaticinare,A,vaticina,nte,re
89,secco,,A,secco,,secco
108,quintupli,quintuplo,A,quintupl,i,o
119,costiera,costiero,A,costier,a,o


In [27]:
train_v.loc[train_v['X_end'] == 'te']

,X,y,PoS,X_beg,X_end,y_end
22,impiastrate,impiastrare,V,impiastra,te,re
54,rieducate,rieducare,V,rieduca,te,re
80,depennate,depennare,V,depenna,te,re
128,graffiate,graffiare,V,graffia,te,re
159,violate,violare,V,viola,te,re
165,autolavate,autolavare,V,autolava,te,re
196,attinte,attingere,V,attin,te,gere
292,imbrattate,imbrattare,V,imbratta,te,re
322,discaricate,discaricare,V,discarica,te,re
431,offendete,offendere,V,offende,te,re


Вернемся пораньше

In [96]:
def sortByLength(inputStr):
        return len(inputStr)
endings = np.unique(train_v[['X_end', 'y']].as_matrix().reshape(-1)).tolist()
endings.sort(key=sortByLength)

In [97]:
exp_data = train_v.copy()
exp_data['X_end_new'] = exp_data['X'].map(lambda x: [x])
for ending in endings:
    exp_data['X_end_new'] = exp_data['X_end_new'].map(lambda x: x + [ending] if len(x[0]) >= len(ending) and x[0][-len(ending):] == ending else x)
exp_data.head()

,X,y,PoS,X_beg,X_end,y_end,X_end_new
37,balenieri,baleniero,A,balenier,i,o,"[balenieri, i]"
48,osanti,osare,A,osa,nti,re,"[osanti, i, nti]"
49,rialzata,rialzato,A,rialzat,a,o,"[rialzata, a]"
55,rivaleggiante,rivaleggiare,A,rivaleggia,nte,re,"[rivaleggiante, e, nte]"
61,cospiranti,cospirare,A,cospira,nti,re,"[cospiranti, i, nti]"


In [103]:
exp_data.loc[exp_data['X_beg']!=exp_data['y_beg']]

,X,y,PoS,X_beg,X_end,y_end,X_end_new,y_beg
4688,massacrante,massacrare,A,massacrant,e,,e,massacrare
11268,qualificante,qualificare,A,qualificant,e,,e,qualificare
15200,restante,restare,A,restant,e,,e,restare
20393,siedevo,soprassedere,A,siedev,o,sedere,o,sopras
28425,opprimente,opprimere,A,oppriment,e,,e,opprimere
32524,maggiori,grande,A,maggior,i,,i,grande
41003,migliori,buono,A,miglior,i,,i,buono
51108,confacente,confare,A,confacent,e,,e,confare
54689,perdente,perdere,A,perdent,e,,e,perdere
62687,maggiore,grande,A,maggior,e,,e,grande


In [104]:
#exp_data['y'] = data.loc[data['PoS']=='A']['y']
new_train_v = exp_data.drop(exp_data.loc[exp_data['X_beg']!=exp_data['y_beg']])

In [105]:
new_train_v

,X,y,PoS,X_beg,X_end,y_end,X_end_new,y_beg
37,balenieri,baleniero,A,balenier,i,o,i,balenier
48,osanti,osare,A,osa,nti,re,nti,osa
49,rialzata,rialzato,A,rialzat,a,o,a,rialzat
55,rivaleggiante,rivaleggiare,A,rivaleggia,nte,re,nte,rivaleggia
61,cospiranti,cospirare,A,cospira,nti,re,nti,cospira
81,turbolenta,turbolento,A,turbolent,a,o,a,turbolent
83,vaticinante,vaticinare,A,vaticina,nte,re,nte,vaticina
89,secco,secco,A,secc,o,o,o,secc
108,quintupli,quintuplo,A,quintupl,i,o,i,quintupl
119,costiera,costiero,A,costier,a,o,a,costier


In [106]:
endings = np.unique(new_train_v[['X_end', 'y_end']].as_matrix().reshape(-1)).tolist()
exp_test = test_v.copy()
exp_test.head()

,X,y,PoS
2,incidentali,incidentali,A
16,impoverimenti,impoverimenti,A
21,sesquipedale,sesquipedale,A
22,timide,timide,A
52,secca,secca,A


In [107]:
exp_test['X_end'] = exp_test['X'].map(lambda x: [x])
for ending in endings:
    exp_test['X_end'] = exp_test['X_end'].map(lambda x: x + [ending] if len(x[0]) >= len(ending) and x[0][-len(ending):] == ending else x)
exp_test.head()

,X,y,PoS,X_end
2,incidentali,incidentali,A,"[incidentali, i]"
16,impoverimenti,impoverimenti,A,"[impoverimenti, enti, i, nti]"
21,sesquipedale,sesquipedale,A,"[sesquipedale, e]"
22,timide,timide,A,"[timide, e]"
52,secca,secca,A,"[secca, a]"


In [108]:
exp_test['X_beg'] = exp_test['X_end'].map(lambda x: x[0][:-len(x[-1])])
exp_test['X_end'] = exp_test['X_end'].map(lambda x: x[-1] if len(x)>1 else x[0][-1])
exp_test = exp_test[['X', 'X_beg', 'X_end']]
exp_test.head()

,X,X_beg,X_end
2,incidentali,incidental,i
16,impoverimenti,impoverime,nti
21,sesquipedale,sesquipedal,e
22,timide,timid,e
52,secca,secc,a


In [109]:
count_vectorizer = CountVectorizer(min_df=5, max_df=.5, max_features=None,ngram_range=(2, 4), analyzer='char_wb', lowercase=False, binary=True)
X_train_form = count_vectorizer.fit_transform(new_train_v['X'])
y_train_form = new_train_v['y_end']

In [55]:
algo = LogisticRegression(penalty='l1', C=0.145)
arr = cross_val_score(algo, X_train_form, y_train_form, cv=2)
arr

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


array([ 0.94162877,  0.9422121 ])

In [110]:
X_test_form = count_vectorizer.transform(exp_test['X'])
algo = LogisticRegression(penalty='l1', C=0.145)
algo.fit(X_train_form, y_train_form)
exp_test['y'] = algo.predict(X_test_form)

In [111]:
exp_test

,X,X_beg,X_end,y
2,incidentali,incidental,i,e
16,impoverimenti,impoverime,nti,ire
21,sesquipedale,sesquipedal,e,e
22,timide,timid,e,o
52,secca,secc,a,o
59,scomponibili,scomponibil,i,e
69,collocabile,collocabil,e,e
72,fornitrici,fornit,rici,ore
85,biascicante,biascica,nte,re
100,dittatorio,dittatori,o,o


In [112]:
for x in exp_test.iterrows():
    if(len(x[1]['X_beg']) > 0 and len(x[1]['y'])>0):
        if(x[1]['X_beg'][-1] == x[1]['y'][0]):
            x[1]['y'] = x[1]['y'][1:]

exp_test

,X,X_beg,X_end,y
2,incidentali,incidental,i,e
16,impoverimenti,impoverime,nti,ire
21,sesquipedale,sesquipedal,e,e
22,timide,timid,e,o
52,secca,secc,a,o
59,scomponibili,scomponibil,i,e
69,collocabile,collocabil,e,e
72,fornitrici,fornit,rici,ore
85,biascicante,biascica,nte,re
100,dittatorio,dittatori,o,o


In [113]:
for i in exp_test.index:
    test['y'][i] = exp_test['X_beg'][i]+exp_test['y'][i]+'+A'
test

,X,y,PoS
1,gettonan,gettonare+V,V
2,incidentali,incidentale+A,A
3,involtino,involtinare+V,V
4,lievi,lievo+N,N
5,comunistizzasse,comunistizzare+V,V
6,vidimerebbe,vidimeare+V,V
7,imbrodan,imbrodare+V,V
8,strillar,strillare+V,V
9,cifrasti,cifrasre+V,V
10,compassavano,compassare+V,V


In [72]:
vvv = exp_test.copy()

In [93]:
nnn = exp_test.copy()

In [114]:
answer = pd.read_csv('task2_lemmas_sample_submission', index_col=0)
answer['Category'] = test['y']
answer

,Category
1,gettonare+V
2,incidentale+A
3,involtinare+V
4,lievo+N
5,comunistizzare+V
6,vidimeare+V
7,imbrodare+V
8,strillare+V
9,cifrasre+V
10,compassare+V


In [115]:
answer.to_csv('task_2_submission', index_label='Id')